In [1]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

'Trains a simple convnet on the MNIST dataset.\n\nGets to 99.25% test accuracy after 12 epochs\n(there is still a lot of margin for parameter tuning).\n16 seconds per epoch on a GRID K520 GPU.\n'

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
print(keras.backend.backend())

tensorflow


In [3]:
import gc
import optuna
import pickle
import time
import shutil

In [4]:
# 乱数固定
SEED = 20241001
keras.utils.set_random_seed(SEED)
import os
os.environ['PYTHONHASHSEED'] = str(SEED)
import numpy as np
np.random.seed(SEED)
import random
random.seed(SEED)
import tensorflow as tf
tf.random.set_seed(SEED)

In [5]:
# 定数
BATCH_SIZE = 128
NUM_CLASSES = 10
EPOCHS = 15
IMG_ROWS, IMG_COLS = 28, 28
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
N_TRIALS = 5
N_LOOPS = 20
SAVE_NAME = 'study_cnn'

In [6]:
# データ生成関数
def make_data():
    # input image dimensions

    # the data, split between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    x_train = x_train.reshape(x_train.shape[0], IMG_ROWS, IMG_COLS, 1)
    x_test = x_test.reshape(x_test.shape[0], IMG_ROWS, IMG_COLS, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print(f'x_train shape: {x_train.shape}')
    print(f'{x_train.shape[0]} train samples')
    print(f'{x_test.shape[0]} test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

    n_train = int(len(x_train)*0.8)

    x_val = x_train[n_train:]
    x_train = x_train[:n_train]

    y_val = y_train[n_train:]
    y_train = y_train[:n_train]

    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

In [7]:
# モデル構築関数
def make_model(n_filter1, n_filter2, dropout):
    inputs = keras.layers.Input(shape=INPUT_SHAPE)
    x = inputs
    x = keras.layers.Conv2D(n_filter1, kernel_size=(3, 3), activation="relu")(x)
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = keras.layers.Conv2D(n_filter2, kernel_size=(3, 3), activation="relu")(x)
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dropout(dropout)(x)
    outputs = keras.layers.Dense(NUM_CLASSES, activation="softmax")(x)
    
    model = keras.models.Model(inputs=inputs, outputs=outputs)

    return model

In [8]:
# 目的関数
def objective(trial):
    
    # エポック終了時に実行(pruneあり)
    def on_epoch_end_prune(epoch, logs={}):
        trial.report(logs.get('loss'), epoch)
        if trial.should_prune():
            print(f'pruned: epoch={epoch}, loss={logs.get("loss")}, val_loss={logs.get("val_loss")}')
            raise optuna.TrialPruned()

    # kerasのセッションをクリア
    keras.backend.clear_session()

    # ハイパーパラメータ生成
    n_filter1 = trial.suggest_int('n_filter1', 16, 128)
    n_filter2 = trial.suggest_int('n_filter2', 32, 256)
    dropout = trial.suggest_float('dropout', 0, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.1)

    # モデル構築
    model = make_model(
        n_filter1=n_filter1,
        n_filter2=n_filter2,
        dropout=dropout,
    )

    # 学習ループ構築
    model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.AdamW(learning_rate=learning_rate, clipnorm=0.001), metrics=["accuracy"])
 
    # 打ち切り判断用のコールバック
    callback = keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end_prune)

    # 学習
    try:
        model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_val, y_val), verbose=0, callbacks=[callback])
    except optuna.TrialPruned as e:
        print(f'TrialPruned: {e}')
        raise optuna.TrialPruned()
    except KeyboardInterrupt:
        print('keyboard interrupted')
        val_loss, val_accuracy = float('nan'), float('nan')
    except Exception as e:
        print(f'Error: {e}')
        val_loss, val_accuracy = float('nan'), float('nan')
    else:
        val_loss, val_accuracy = model.evaluate(x_val, y_val, verbose=0)
    finally:
        # 掃除
        del model
        keras.backend.clear_session()
        gc.collect

    print(f'{val_loss=}, {val_accuracy=}')

    # 結果を返す(val_accuracy最大化を目指す)
    return val_accuracy

In [9]:
# データ生成
(x_train, y_train), (x_val, y_val), (x_test, y_test) = make_data()

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [10]:
# 最適化の準備
sampler = optuna.samplers.TPESampler(seed=SEED)
pruner = optuna.pruners.HyperbandPruner(min_resource=1, max_resource=EPOCHS, reduction_factor=3)
study = optuna.create_study(sampler=sampler, pruner=pruner, direction='maximize')

[I 2025-05-22 23:17:10,422] A new study created in memory with name: no-name-f8a49310-a9ef-47b4-856d-22ac062f9c9e


In [11]:
# 開始時刻記録
start_time = time.time()

In [12]:
# 最適化実行
for i in range(N_LOOPS):
    # ハイパーパラメータ最適化
    study.optimize(objective, n_trials=N_TRIALS)

    sum_trials = N_TRIALS*(i+1)

    # ハイパーパラメータ最適化の結果
    print(f'best_params={study.best_params} # trials={sum_trials}, best_value={study.best_value}')
    best_params = study.best_params

    # 結果保存
    with open(f'{SAVE_NAME}{sum_trials:04d}.pkl', 'wb') as f:
        pickle.dump(study, f)

    # 古いシンボリックリンクがあれば削除
    if os.path.islink('study.pkl'):
        os.remove('study.pkl')

    try:
        # 最新の結果をstudy.pklとしてシンボリックリンクを作っておく
        os.symlink(f'{SAVE_NAME}{sum_trials:04d}.pkl', 'study.pkl')
    except Exception as e:
        # シンボリックリンクがだめならコピーする
        shutil.copy2(f'{SAVE_NAME}{sum_trials:04d}.pkl', 'study.pkl')

[I 2025-05-22 23:19:11,566] Trial 0 finished with value: 0.9869166612625122 and parameters: {'n_filter1': 24, 'n_filter2': 100, 'dropout': 0.05361584483320092, 'learning_rate': 0.018494386961884945}. Best is trial 0 with value: 0.9869166612625122.


val_loss=0.16511759161949158, val_accuracy=0.9869166612625122


[I 2025-05-22 23:21:58,997] Trial 1 finished with value: 0.9007499814033508 and parameters: {'n_filter1': 39, 'n_filter2': 96, 'dropout': 0.23802507836915165, 'learning_rate': 0.09959959584909425}. Best is trial 0 with value: 0.9869166612625122.


val_loss=0.31111904978752136, val_accuracy=0.9007499814033508


[I 2025-05-22 23:35:14,567] Trial 2 finished with value: 0.921916663646698 and parameters: {'n_filter1': 100, 'n_filter2': 158, 'dropout': 0.4727125650511506, 'learning_rate': 0.08663718979808867}. Best is trial 0 with value: 0.9869166612625122.


...

pruned: epoch=1, loss=0.3032459020614624, val_loss=0.3073215186595917
TrialPruned: 


[I 2025-05-23 03:22:16,842] Trial 96 pruned. 
[I 2025-05-23 03:24:30,080] Trial 97 finished with value: 0.9072499871253967 and parameters: {'n_filter1': 38, 'n_filter2': 94, 'dropout': 0.32147821426141, 'learning_rate': 0.09160503836504835}. Best is trial 0 with value: 0.9869166612625122.


val_loss=0.33608394861221313, val_accuracy=0.9072499871253967
pruned: epoch=9, loss=0.39708268642425537, val_loss=0.14592228829860687
TrialPruned: 


[I 2025-05-23 03:26:26,476] Trial 98 pruned. 
[I 2025-05-23 03:27:12,774] Trial 99 pruned. 


pruned: epoch=1, loss=0.3513062000274658, val_loss=0.16825664043426514
TrialPruned: 
best_params={'n_filter1': 24, 'n_filter2': 100, 'dropout': 0.05361584483320092, 'learning_rate': 0.018494386961884945} # trials=100, best_value=0.9869166612625122


In [13]:
# 所要時間表示
elapsed_time = time.time() - start_time
print(f'elapsed time: {elapsed_time}[sec]')

elapsed time: 15002.350652217865[sec]


In [14]:
# 結果表示
print(f'best_params: {study.best_params}')
print(f'best_value: {study.best_value}')

best_params: {'n_filter1': 24, 'n_filter2': 100, 'dropout': 0.05361584483320092, 'learning_rate': 0.018494386961884945}
best_value: 0.9869166612625122
